In [1]:
%pip install PyMuPDF


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pytesseract
# https://codetoprosper.com/tesseract-ocr-for-windows

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import fitz
import pytesseract
import pandas as pd
from PIL import Image
import io
import cv2
import numpy as np
from datetime import datetime

In [5]:
def extract_images_and_text(pdf_path, output_folder, text_window=100):
    # Create an empty list to store image and text information
    image_text_info_list = []

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Set up pytesseract
    pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'  # Change to your Tesseract executable path

    for page_number, page in enumerate(pdf_document):
        # Get all the images on the current page
        images = page.get_images(full=True)
        print(f"page number: {page_number}")
        print(f"Image details: {images}")
    
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            image_data = base_image["image"]

            # Determine the file extension based on the image type
            if base_image["ext"] == ".jpg":
                image_ext = "jpg"
            elif base_image["ext"] == ".png":
                image_ext = "png"
            else:
                image_ext = "png"

            # Create a dictionary with image information
            image_info = {
                "File_Name": os.path.basename(pdf_path),
                "Image_Name": f"page_{page_number + 1}_image_{img_index + 1}.{image_ext}",
                "PageNumber": page_number + 1
            }

            # Extract text content before and after the image
            text_before = page.get_text("text", clip=(0, 0, img[2], img[3]-text_window))
            text_after = page.get_text("text", clip=(0, img[3], img[2], img[3]+text_window))

            # Convert image data to a format that Tesseract supports
            image_pil = Image.open(io.BytesIO(image_data))
            image_data = image_pil.convert("RGB")

            # Use OCR to extract text inside the image
            image_text = pytesseract.image_to_string(image_data)

            image_info["Text_Before"] = text_before.strip()
            image_info["Text_After"] = text_after.strip()
            image_info["Text_Inside_Image"] = image_text.strip()
            

            # Create the output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)

            # Save the image to the output folder
            image_filename = os.path.join(output_folder, image_info["Image_Name"])
            image_pil.save(image_filename, format=image_ext)
            #with open(image_filename, "wb") as image_file:
            #    image_file.write(image_pil.tobytes())
            
            # Append the image and text information to the list
            image_text_info_list.append(image_info)

    pdf_document.close()

    # Create a Pandas DataFrame from the image and text information list
    df = pd.DataFrame(image_text_info_list)
    return df

In [6]:
pdf_files = os.listdir("C:/Thesis/CSR_Reports/")
print(pdf_files)
# Reading all the files and extract images 
count = 0

damage_list = []
total_df=[]
for pdf in pdf_files:
    count = count+1
    print("start time", datetime.now() )
    print("count:", count)
    print("pdf file:", pdf)
    #pdffile = "CSR Reports/"+pdf
    
    pdf_file = "C:/Thesis/CSR_Reports/" + pdf  # PDF file path
    output_dir = "C:/Thesis/Images/" + pdf[:-4]  #
    #output_dir = "1time_2011"  # Replace with the output folder paths
    
    try:
        df = extract_images_and_text(pdf_file, output_dir)
        if count == 1:
            total_df = df
        else:
            total_df = total_df.append(df,ignore_index=True) 
      
    except:
        damage_list.append(pdf)
    
    print("end time", datetime.now() )

['3M_2017.pdf']
start time 2023-11-12 20:32:34.500248
count: 1
pdf file: 3M_2017.pdf
page number: 0
Image details: []
page number: 1
Image details: [(6, 0, 901, 491, 8, 'ICCBased', '', 'Im0', 'DCTDecode', 0), (8, 0, 329, 247, 8, 'ICCBased', '', 'Im1', 'DCTDecode', 0), (10, 0, 1229, 834, 8, 'ICCBased', '', 'Im2', 'DCTDecode', 0), (12, 0, 328, 249, 8, 'ICCBased', '', 'Im3', 'DCTDecode', 0), (14, 0, 324, 273, 8, 'ICCBased', '', 'Im4', 'DCTDecode', 0), (16, 0, 308, 272, 8, 'ICCBased', '', 'Im5', 'DCTDecode', 0), (18, 0, 306, 268, 8, 'ICCBased', '', 'Im6', 'DCTDecode', 0), (20, 0, 304, 272, 8, 'ICCBased', '', 'Im7', 'DCTDecode', 0)]
page number: 2
Image details: [(26, 0, 509, 678, 8, 'ICCBased', '', 'Im0', 'DCTDecode', 0), (28, 0, 358, 437, 8, 'ICCBased', '', 'Im1', 'DCTDecode', 0), (30, 0, 354, 441, 8, 'ICCBased', '', 'Im2', 'DCTDecode', 0), (32, 0, 358, 388, 8, 'ICCBased', '', 'Im3', 'DCTDecode', 0), (34, 0, 358, 441, 8, 'ICCBased', '', 'Im4', 'DCTDecode', 0), (36, 0, 358, 437, 8, 'ICCBas

In [7]:
print(total_df)

                 File_Name            Image_Name  PageNumber  \
0              3M_2017.pdf    page_2_image_1.png           2   
1              3M_2017.pdf    page_2_image_2.png           2   
2              3M_2017.pdf    page_2_image_3.png           2   
3              3M_2017.pdf    page_2_image_4.png           2   
4              3M_2017.pdf    page_2_image_5.png           2   
...                    ...                   ...         ...   
7668  FrankeGroup_2012.pdf  page_26_image_11.png          26   
7669  FrankeGroup_2012.pdf  page_26_image_12.png          26   
7670  FrankeGroup_2012.pdf  page_26_image_13.png          26   
7671  FrankeGroup_2012.pdf  page_26_image_14.png          26   
7672  FrankeGroup_2012.pdf   page_28_image_1.png          28   

                                            Text_Before  \
0     Improving every life\nA desirable location for...   
1                                                         
2     Improving every life\nIt takes a community\nSe..

In [ ]:
#df = extract_images_and_text(pdf_file, output_dir)

# Print the DataFrame (optional)


In [8]:
tatal_df.columns

Index(['File_Name', 'Image_Name', 'PageNumber', 'Text_Before', 'Text_After',
       'Text_Inside_Image'],
      dtype='object')

In [9]:
tatal_df.head()

,File_Name,Image_Name,PageNumber,Text_Before,Text_After,Text_Inside_Image
0,3M_2017.pdf,page_2_image_1.png,2,Improving every life\nA desirable location for...,It takes a community\n3M Legal Affairs pro bon...,
1,3M_2017.pdf,page_2_image_2.png,2,,,
2,3M_2017.pdf,page_2_image_3.png,2,Improving every life\nIt takes a community\nSe...,,
3,3M_2017.pdf,page_2_image_4.png,2,,,oe “” This isn’t the end.\nr It’s just the beg...
4,3M_2017.pdf,page_2_image_5.png,2,,,


In [ ]:
tatal_df.to_csv('SaveFiles/ExtractedData.csv', index=False)